In [28]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import time
from bs4 import BeautifulSoup
import json
import datetime

In [3]:
# data_from_excel = pd.read_excel("Chốt số liệu Thanh Toán cho INFLU.xlsx", sheet_name="View", skiprows = 2, index_col= None)
sheet_id = '1a6oSAow7ZprMsGbbOv-bXEPJHo3y8nmPX5wimpyq1VQ'
sheet_name = 'View'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}' 
data_from_excel = pd.read_csv(url)

In [4]:
data_from_excel.reset_index(inplace=True)

In [6]:
data = data_from_excel

In [9]:
data.dropna(subset=["Link Video"], inplace=True)
Tiktok_data = data[data['Link Video'].str.contains("tiktok.com")]

# Tiktok

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.tiktok.com/login/")

In [18]:
begin_time = datetime.datetime.now()

viewcount_list = []

for i in Tiktok_data["Link Video"].values:
    url = ('view-source:' + i)
    driver.get(url)
    try:
        data_j = driver.find_element_by_tag_name('body').text
        soup = BeautifulSoup(data_j)

        info = soup.find("script", type="application/json").contents[0]
        json_data = json.loads(info)

        view = json_data["props"]["pageProps"]["itemInfo"]["itemStruct"]["stats"]["playCount"]

        item = {    
                    "Link Video": i,
                    "Số View": view,
        }

        viewcount_list.append(item)
    except:
        print(i)

df = pd.DataFrame(viewcount_list)
df = Tiktok_data.merge(df, on="Link Video", how="left")  
Tiktok_time = begin_time
df.to_pickle("./tiktok.pkl")

print(datetime.datetime.now() - begin_time)

https://www.tiktok.com/@thuycute05123/video/6985715429998185754?lang=vi-VN&is_copy_url=0&is_from_webapp=v1&sender_device=pc&sender_web_id=6992511724982355458
https://www.tiktok.com/ZGJAJB8Ue/
https://www.tiktok.com/@thuonghienn/video/6996620377145232666?
https://www.tiktok.com/@meous99/video/6997343418158419227?
https://www.tiktok.com/@thuhuong2xx1/video/6996979355477871899?is_copy_url=0&is_from_webapp=v1&sender_device=pc&sender_web_id=6983582057944237570
0:10:04.502749


# Instagram

In [22]:
insta_data = data[data['Link Video'].str.contains('instagram')]

# get rid of ?utm_...
insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]

<ipython-input-22-3b6a3fb73241>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]


In [23]:
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/accounts/login/")

In [24]:
insta_list = []

for link in insta_data["Link Video"].values:
    base_url = link
    url_json = (base_url + '?__a=1')
    driver.get(url_json)
    
    data_j = driver.find_element_by_tag_name('body').text
    try:
        data_json = json.loads(data_j)

        view = data_json["graphql"]['shortcode_media']['video_view_count']

        item = {
                    "Link Video": link,
                    "Số View": view,
        }

        insta_list.append(item)
        time.sleep(.5)
    except:
        print(link + " Cannot get insta info!")
    
df = pd.DataFrame(insta_list)
df = insta_data.merge(df, on="Link Video", how="left")
df.to_pickle("insta.pkl")
insta_datetime = datetime.datetime.now()

# Facebook

In [25]:
fb_data = data[data['Link Video'].str.startswith('https://www.facebook.com/')]
fb_data = fb_data[fb_data['Link Video'].str.contains("/videos/")]

In [74]:
driver = webdriver.Chrome()
driver.get("https://www.facebook.com/")

In [75]:
fb_list = []

for link in fb_data["Link Video"].values:
 
    driver.get(link)
    
    try:
        view_count = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('//span[@class= "_26fq"]').text)

        if(view_count.find("triệu") == -1): # normal K count
            view = float(view_count[:-10].replace(",","."))
        else:
            view = int(view_count[:2]) * 1000 # million count, take 1 or 2 first digit times 10000000

        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        fb_list.append(item)
    
    except:
        print(link)
        
df = pd.DataFrame(fb_list)
df = fb_data.merge(df, on="Link Video", how="left")
df.to_pickle("fb.pkl")

fb_datetime = datetime.datetime.now()

In [76]:
tiktok = pd.read_pickle("tiktok.pkl")
insta = pd.read_pickle("insta.pkl")
fb = pd.read_pickle("fb.pkl")

In [86]:
fb["Số View"].values

array([  11. ,   52. ,    3. ,  104. ,  100. ,    2.4,  102. ,    2.6,
         33. ,   50. ,   51. ,   52. ,   21. ,  102. ,   20. ,   10. ,
         51. ,    4.2,   54. ,    5.2,    5.8,   98. ,  663. ,    3.6,
         90. ,  102. ,   50. ,   50. ,   55. ,    5.5,  104. ,    3.9,
         41. ,   55. ,   33. ,  271. , 1000. ,  412. ,  307. ,   97. ,
        158. ,   17. ,   14. ,  306. ,   48. ,    2.3,   54. ,  347. ,
         50. ,   52. ,   11. ,    2.9,   60. ,  102. ,   50. ,  100. ,
         53. ,    2.8,   51. ,   11. ,    5.8,    4.1,  102. ,    3.5,
         72. ])

In [78]:
tiktok_insta = pd.concat([tiktok, insta])
tiktok_insta["Số View"] = tiktok_insta["Số View"] / 1000
tiktok_insta_fb = pd.concat([tiktok_insta, fb])

In [79]:
tiktok_insta_fb = tiktok_insta_fb[["index", "Số View"]]

In [80]:
export_df = data_from_excel.merge(tiktok_insta_fb, on="index", how="left")

In [82]:
time_df = pd.DataFrame({"Ngày giờ" :["Tiktok : " + str(Tiktok_time), "Insta : " + str(insta_datetime), "Fb : " + str(fb_datetime)]})
export_df = pd.concat([export_df, time_df], axis=1)
export_df.rename(columns={"Số View": "Số view (đơn vị 1000)"}, inplace=True)
push_df = export_df[["STT", "Mã INF", "Kênh hợp tác", "Link Video", "Người phụ trách", "Số view (đơn vị 1000)", "Ngày giờ"]]

In [83]:
final_push = push_df[~push_df.duplicated()]

In [84]:
final_push

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Số view (đơn vị 1000),Ngày giờ
0,1.0,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...,NaN,11.0,Tiktok : 2021-08-19 10:19:21.217716
1,2.0,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,NaN,122.9,Insta : 2021-08-19 10:38:34.422582
2,3.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,NaN,42.3,Fb : 2021-08-19 12:17:39.661319
3,4.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,NaN,63.3,NaN
4,5.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,NaN,101.4,NaN
...,...,...,...,...,...,...,...
512,339.0,INFHUONG558,TikTok,https://www.tiktok.com/@huongmin_/video/698919...,Mến,61.0,NaN
514,340.0,INFHUONG558,TikTok,https://www.tiktok.com/@huongmin_/video/698955...,Mến,426.4,NaN
516,341.0,INFQUYEN983,TikTok,https://www.tiktok.com/@kmquyen2002/video/6989...,Mến,361.0,NaN
518,342.0,INFPHUONG332,TikTok,https://www.tiktok.com/@vuhphuong/video/698910...,Mến,2000.0,NaN


In [85]:
import gspread
from gspread_dataframe import set_with_dataframe

# ACCES GOOGLE SHEET
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('Python') #->gg sheet name 

# APPEND DATA TO SHEET
your_dataframe = final_push
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET